Not appropriate to use integrated values for differential genes
https://github.com/satijalab/seurat/issues/5881
Label mouse and human with macaque names and then try finding differential genes for each dataset

# 1. Create and save Marker Genes

In [1]:
## conda activate r4
## packages for data table processing

#IRkernel::installspec()

## main Seurat package snRNA-seq packages
library(Seurat)
library(SeuratDisk)
library(future)

library(here)
library(tidyverse)

library(caret)
library(cluster)

#install.packages('hash')
library(hash)

ss <- function(x, pattern, slot = 1, ...) { 
  sapply(strsplit(x = x, split = pattern, ...), '[', slot) }
options(stringsAsFactors = F)
options(repr.plot.width=11, repr.plot.height=8.5)

#######################################################
# 0) Seurat uses the future package for parallelization
## set to be parallel over 28 cores
plan("multicore", workers = 28)
options(future.globals.maxSize = 100 * 1024^3)
options(future.rng.onMisuse = 'ignore')

Warning message:
“package ‘Seurat’ was built under R version 4.1.3”
Attaching SeuratObject

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Warning message:
“package ‘future’ was built under R version 4.1.3”
Warning message:
“package ‘here’ was built under R version 4.1.3”
here() starts at /projects/pfenninggroup/singleCell/Macaque_SealDorsalHorn_snATAC-seq

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘ggplot2’ was built under R version 4.1.3”
Warning message:
“package ‘tibble’ was built under R version 4.1.3”
Warning message:
“package ‘dplyr’ was built under R version 4.1.3”
── Conflicts ──────────────────────────────────────────

In [2]:
# load human and macaque rna dataset
macaque_rna = LoadH5Seurat("/projects/pfenninggroup/singleCell/Macaque_SealDorsalHorn_snATAC-seq/data/tidy_data/rdas/macaque_integrated_neuronsv3.h5seurat",assay = "RNA")
macaque_rna$nCount_RNA = macaque_rna$nUMI # equivalent data
macaque_rna$nFeature_RNA = macaque_rna$nGene # equivalent data

human_rna = readRDS('/projects/pfenninggroup/singleCell/HumanMacaqueMouse_DorsalHorn_snRNA-Seq/data/tidy_data/Levine_Human_dorsal_horn_celltypes/rdas/human_neurons.rds')

mouse_rna = LoadH5Seurat('/home/bnphan/projects/2022_DH_Russ_cell_types/data/tidy_data/rdas/Russ_et_al_dorsal_horn_neurons.h5seurat')
mouse_rna = mouse_rna[, mouse_rna$dataset == 'Sathyamurthy']
Idents(object = mouse_rna) <- "final_cluster_assignment"
DefaultAssay(mouse_rna) = 'RNA'

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding reduction pca

Adding cell embeddings for pca

Adding miscellaneous information for pca

Adding reduction scanorama

Adding cell embeddings for scanorama

Adding miscellaneous information for scanorama

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding command information

Adding cell-level metadata

Validating h5Seurat file

Initializing MAGIC_RNA with data

Adding miscellaneous information for MAGIC_RNA

Initializing MAGIC_SCT with data

Adding miscellaneous information for MAGIC_SCT

Initializing RNA with data

Adding counts for RNA

Adding miscellaneous information for RNA

Initializing SCT with data

Adding counts for SCT

Adding scale.data for SCT

Adding variable feature information for SCT

Adding miscellaneous information for SCT

Initializing alra with data

Adding miscellaneous information for alra


In [3]:
### mouse genes to human genes
convertHumanGeneList <- function(x){
  file = '/home/bnphan/resources/genomes/GRCh38.p13/ENSEMBL_GRCh38.p13_genes_to_Orthologous_mouse_genes.tsv'
  orthologs = read_tsv(file, show_col_types = FALSE) %>% 
    rename_with(make.names) %>% 
    dplyr::filter(Mouse.homology.type== 'ortholog_one2one') %>% 
    dplyr::filter(Mouse.gene.name %in% x) %>% 
    dplyr::select(Mouse.gene.name, Gene.name)
  
  # Print the first 6 genes found to the screen
  print(head(orthologs))
  return(orthologs)
}
# RenameGenesSeurat from https://github.com/satijalab/seurat/issues/1049
RenameGenesSeurat <- function(obj, newnames) {
  # Replace gene names in different slots of a Seurat object. Run this before integration.
  # It only changes obj@assays$RNA@counts, @data and @scale.data.
  print("Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data.")
  RNA <- obj@assays$RNA

  if (nrow(RNA) == length(newnames)) {
    if (length(RNA@counts)) RNA@counts@Dimnames[[1]]            <- newnames
    if (length(RNA@data)) RNA@data@Dimnames[[1]]                <- newnames
    if (length(RNA@scale.data)) RNA@scale.data@Dimnames[[1]]    <- newnames
  } else {"Unequal gene sets: nrow(RNA) != nrow(newnames)"}
  obj@assays$RNA <- RNA
  return(obj)
}

## switch out human gene names w/ orthologous mouse gene names, based on ENSEMBL
MmtoHg = convertHumanGeneList(rownames(mouse_rna)) %>% deframe()
HgtoMm = setNames(names(MmtoHg), MmtoHg)
# mouse_rna = mouse_rna[names(MmtoHg[!is.na(MmtoHg)]), ]
# mouse_rna = RenameGenesSeurat(mouse_rna, newnames = MmtoHg[rownames(mouse_rna)])
# mouse_rna = mouse_rna[!is.na(rownames(mouse_rna)), ]
# head(rownames(mouse_rna))

# firstup <- function(x) {
#   substr(x, 1, 1) <- toupper(substr(x, 1, 1))
#   x
# }

# A tibble: 6 × 2
  Mouse.gene.name Gene.name
  <chr>           <chr>    
1 Usp9y           USP9Y    
2 Kdm5d           KDM5D    
3 Sry             SRY      
4 Zfx             ZFY      
5 Ddx3y           DDX3Y    
6 Uty             UTY      


In [5]:
## subset to dorsal horn
human_cell_types = unique(human_rna$final_ann)
human_dh_types = human_cell_types[grep('Dorsal', human_cell_types)]

macaque_cell_types = unique(macaque_rna$cluster_type)
GLUTs = macaque_cell_types[grep('GLUT', macaque_cell_types)]
GABAs = macaque_cell_types[grep('GABA', macaque_cell_types)]
macaque_dh_types = c(GLUTs, GABAs)

Idents(object = human_rna) <- "final_ann"
human_rna_dh = subset(x = human_rna, idents = human_dh_types)
rm(human_rna)

Idents(object = macaque_rna) <- "cluster_type"
macaque_rna_dh = subset(x = macaque_rna, idents = macaque_dh_types)
rm(macaque_rna)

ERROR: Error in unique(human_rna$final_ann): object 'human_rna' not found


In [7]:
date = '2023-08-16'
human_to_macaque_predictions = readRDS(file = here(paste0("data/tidy_data/rdas/macaque_labels_for_human",date,".rds")))
mouse_to_macaque_predictions = readRDS(file = here(paste0("data/tidy_data/rdas/macaque_labels_for_mouse",date,".rds")) )

mouse_rna <- AddMetaData(object = mouse_rna, metadata = mouse_to_macaque_predictions)
human_rna_dh <- AddMetaData(object = human_rna_dh, metadata = human_to_macaque_predictions)

Idents(object = human_rna_dh) <- "predicted.id"
Idents(object = mouse_rna) <- "predicted.id"

In [7]:
x = rowSums(GetAssayData(object = macaque_rna_dh, slot = "counts"))
macaque_expressed_genes = names(x[x > 0])

x = rowSums(GetAssayData(object = mouse_rna, slot = "counts"))
mouse_expressed_genes = names(x[x > 0])

x = rowSums(GetAssayData(object = human_rna_dh, slot = "counts"))
human_expressed_genes = names(x[x > 0])

### save all expressed genes -- used as reference for GO
write.csv(macaque_expressed_genes, here(paste0('data/tidy_data/tables/go_analysis/macaque_expressed_genes',as.character(Sys.Date()),'.csv')),row.names = FALSE  )
write.csv(mouse_expressed_genes, here(paste0('data/tidy_data/tables/go_analysis/mouse_expressed_genes',as.character(Sys.Date()),'.csv')),row.names = FALSE  )
write.csv(human_expressed_genes, here(paste0('data/tidy_data/tables/go_analysis/human_expressed_genes',as.character(Sys.Date()),'.csv')),row.names = FALSE  )

In [12]:
# ### make cell-specific tables

# # macaque:
# for (cluster_type in unique(all.markers.macaque$cluster)){
#     x = all.markers.macaque[all.markers.macaque$cluster == cluster_type,]
#     x = x[x$avg_log2FC > 0,]
#     genes = x$gene
#     file_name = here(paste0('data/tidy_data/tables/go_analysis/','macaque_',cluster_type,'specific_genes053123.csv'))
#     write.csv(genes, file_name,row.names = FALSE  )
# }
# # mouse:
# for (cluster_type in unique(all.markers.mouse.macaqueLabels$cluster)){
#     x = all.markers.mouse.macaqueLabels[all.markers.mouse.macaqueLabels$cluster == cluster_type,]
#     x = x[x$avg_log2FC > 0,]
#     genes = x$gene
#     file_name = here(paste0('data/tidy_data/tables/go_analysis/','mouse_',cluster_type,'specific_genes053123.csv'))
#     write.csv(genes, file_name,row.names = FALSE  )
# }
# # mouse:
# for (cluster_type in unique(all.markers.human.macaqueLabels$cluster)){
#     x = all.markers.human.macaqueLabels[all.markers.human.macaqueLabels$cluster == cluster_type,]
#     x = x[x$avg_log2FC > 0,]
#     genes = x$gene
#     file_name = here(paste0('data/tidy_data/tables/go_analysis/','human_',cluster_type,'specific_genes053123.csv'))
#     write.csv(genes, file_name,row.names = FALSE  )  
# }


# write.csv(all.markers.mouse.macaqueLabels, here('data/tidy_data/tables/mouse_marker_genes_macaqueLabels052223.csv'),row.names = FALSE  )
# write.csv(all.markers.human.macaqueLabels, here('data/tidy_data/tables/human_marker_genes_macaqueLabels052223.csv'),row.names = FALSE  )

In [10]:
### marker gene lists (based on transfer labeling)
### Find sets of marker genes for all cell types for all three datasets

all.markers.macaque <- FindAllMarkers(object = macaque_rna_dh)
print('macaque done')
all.markers.mouse <- FindAllMarkers(object = mouse_rna)
print('mouse done')
all.markers.human <- FindAllMarkers(object = human_rna_dh)
print('human done')


write_over_markers = FALSE
if (write_over_markers){
write.csv(all.markers.macaque, here(paste0('data/tidy_data/tables/macaque_marker_genes',as.character(Sys.Date()),'.csv')),row.names = FALSE  )
write.csv(all.markers.mouse, here(paste0('data/tidy_data/tables/mouse_marker_genes',as.character(Sys.Date()),'.csv')),row.names = FALSE  )
write.csv(all.markers.human, here(paste0('data/tidy_data/tables/human_marker_genes',as.character(Sys.Date()),'.csv')),row.names = FALSE  )
    
    }

Calculating cluster GLUT1



In [37]:
### filtering for logFC > 0.5 and renaming mouse genes
macaque_for_xspecies = all.markers.macaque[all.markers.macaque$avg_log2FC > 0.3,]
mouse_for_xspecies = all.markers.mouse[all.markers.mouse$avg_log2FC > 0.3,]
# rename genes in mouse
mouse_for_xspecies$gene = as.character(MmtoHg[mouse_for_xspecies$gene])
human_for_xspecies = all.markers.human[all.markers.human$avg_log2FC > 0.3,]

### find cross-species marker genes
macaque_human = merge(x = macaque_for_xspecies, y = human_for_xspecies, by = c("gene","cluster"), suffixes = c(".macaque",".human"))
macaque_human_mouse = merge(x = macaque_human, y = mouse_for_xspecies, by = c("gene","cluster"), suffixes = c("",".mouse"))

## get genes that only appear for a single cell type, especially unique
cell_type_specific_genes = names(table(macaque_human_mouse$gene)[table(macaque_human_mouse$gene) == 1])
macaque_human_mouse_specific = macaque_human_mouse[macaque_human_mouse$gene %in% cell_type_specific_genes,]

In [38]:
#### save tables
filename1 = here(paste0('data/tidy_data/tables/xspecies_marker_genes',as.character(Sys.Date()),'.csv'))
filename2 = here(paste0('data/tidy_data/tables/xspecies_unique_marker_genes',as.character(Sys.Date()),'.csv'))
write.csv(macaque_human_mouse, filename1, row.names = FALSE  )
write.csv(macaque_human_mouse_specific, filename2 ,row.names = FALSE  )

In [63]:
### just macaque and mouse (for glut11)
macaque_mouse = merge(x = macaque_for_xspecies, y = mouse_for_xspecies, by = c("gene","cluster"), suffixes = c(".macaque",".mouse"))
## get genes that only appear for a single cell type, especially unique
macaque_mouse_specific_genes = names(table(macaque_mouse$gene)[table(macaque_mouse$gene) == 1])
macaque_mouse_specific = macaque_mouse[macaque_mouse$gene %in% macaque_mouse_specific_genes,]

In [64]:
#### save macaque mouse
filename1 = here(paste0('data/tidy_data/tables/macaque_mouse_marker_genes',as.character(Sys.Date()),'.csv'))
filename2 = here(paste0('data/tidy_data/tables/macaque_mouse_unique_marker_genes',as.character(Sys.Date()),'.csv'))
write.csv(macaque_mouse, filename1, row.names = FALSE  )
write.csv(macaque_mouse_specific, filename2 ,row.names = FALSE  )

In [17]:
### glut10 versus similar cell types
## find  markers that differentiate GABA4_1 and GABA4_2
# find GABA4_1 markers
GLUT10.markers.macaque <- FindMarkers(object = macaque_rna_dh, ident.1 = "GLUT10", ident.2 = "GLUT8", group.by = 'cluster_type')
print('macaque done')
GLUT10.markers.mouse <- FindMarkers(object = mouse_rna, ident.1 = "GLUT10", ident.2 = "GLUT8", group.by = 'predicted.id')
print('mouse done')
GLUT10.markers.human <- FindMarkers(object = human_rna_dh, ident.1 = "GLUT10", ident.2 = "GLUT8", group.by = 'predicted.id')
print('human done')

[1] "macaque done"
[1] "mouse done"
[1] "human done"


In [12]:
### just GLUT4
GLUT4.markers.macaque <- FindMarkers(object = macaque_rna_dh, ident.1 = "GLUT4", group.by = 'cluster_type')
head(GLUT4.markers.macaque[order(GLUT4.markers.macaque$avg_log2FC, decreasing = TRUE),])

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RASGRF2,7.861461e-24,0.9347410,0.959,0.468,4.315471e-19
TTC39C,1.659773e-25,0.9341314,1.000,0.600,9.111160e-21
AC002070.1,1.207009e-19,0.8475092,0.939,0.412,6.625757e-15
PLCH1,2.649383e-20,0.8285006,0.878,0.400,1.454353e-15
MEIS2,1.386191e-20,0.7990623,0.592,0.144,7.609359e-16
COL14A1,5.811533e-25,0.7844630,0.776,0.227,3.190183e-20


In [20]:
## find  markers that differentiate GABA4_1 and GABA4_2
# find GABA4_1 markers
GABA4_1.markers.macaque <- FindMarkers(object = macaque_rna_dh, ident.1 = "GABA4_1", ident.2 = "GABA4_2", group.by = 'cluster_type')
print('macaque done')
GABA4_1.markers.mouse <- FindMarkers(object = mouse_rna, ident.1 = "GABA4_1", ident.2 = "GABA4_2", group.by = 'predicted.id')
print('mouse done')
GABA4_1.markers.human <- FindMarkers(object = human_rna_dh, ident.1 = "GABA4_1", ident.2 = "GABA4_2", group.by = 'predicted.id')
print('human done')

[1] "macaque done"
[1] "macaque done"
[1] "mouse done"
[1] "human done"


In [6]:
## find  markers that differentiate GABA2_1 and GABA2_2
# find GABA2_1 markers
GABA2_1.markers.macaque <- FindMarkers(object = macaque_rna_dh, ident.1 = "GABA2_1", ident.2 = "GABA2_2", group.by = 'cluster_type')
print('macaque done')
GABA2_1.markers.mouse <- FindMarkers(object = mouse_rna, ident.1 = "GABA2_1", ident.2 = "GABA2_2", group.by = 'predicted.id')
print('mouse done')
GABA2_1.markers.human <- FindMarkers(object = human_rna_dh, ident.1 = "GABA2_1", ident.2 = "GABA2_2", group.by = 'predicted.id')
print('human done')

[1] "macaque done"
[1] "mouse done"
[1] "human done"


In [7]:
## find  markers that differentiate GABA2_1 and GABA2_2
# find GABA2_1 markers
GABA2_2.markers.macaque <- FindMarkers(object = macaque_rna_dh, ident.1 = "GABA2_2", ident.2 = "GABA2_1", group.by = 'cluster_type')
print('macaque done')
GABA2_2.markers.mouse <- FindMarkers(object = mouse_rna, ident.1 = "GABA2_2", ident.2 = "GABA2_1", group.by = 'predicted.id')
print('mouse done')
GABA2_2.markers.human <- FindMarkers(object = human_rna_dh, ident.1 = "GABA2_2", ident.2 = "GABA2_1", group.by = 'predicted.id')
print('human done')

[1] "macaque done"
[1] "mouse done"
[1] "human done"


In [8]:
head(GABA2_1.markers.macaque[order(GABA2_1.markers.macaque$avg_log2FC, decreasing = TRUE),])

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RORB,9.006204e-19,1.3489386,0.846,0.127,4.943866e-14
SDK1,1.154755e-16,1.0832012,0.938,0.493,6.338913e-12
SORCS3,1.316942e-12,0.9441789,0.769,0.239,7.229220e-08
GRIK3,2.125699e-11,0.8198559,0.615,0.085,1.166881e-06
SILC1,5.547156e-11,0.7576759,0.815,0.282,3.045056e-06
CLSTN2,1.626375e-07,0.7056366,0.846,0.437,8.927822e-03


In [18]:
GABA4_1.markers.macaque[order(GABA4_1.markers.macaque$avg_log2FC, decreasing = FALSE),]

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SDK2,3.613456e-26,-1.4240033,0.275,0.93,1.983570e-21
PTPRD,5.822688e-19,-1.0249427,0.516,0.89,3.196306e-14
NKAIN3,2.886578e-17,-1.0017224,0.330,0.81,1.584558e-12
NKAIN2,1.791046e-14,-0.9220274,0.505,0.80,9.831766e-10
SGCD,1.494559e-14,-0.9080698,0.396,0.82,8.204230e-10
SNTG1,1.654332e-16,-0.8734657,0.615,0.88,9.081291e-12
NTNG1,1.615332e-13,-0.8690139,0.264,0.69,8.867204e-09
PKNOX2,2.054437e-18,-0.7849692,0.802,0.98,1.127762e-13
TNR,9.238750e-15,-0.7623030,0.187,0.72,5.071519e-10


In [27]:
GABA4_2.vsAllmarkers.mouse <- FindMarkers(object = mouse_rna, ident.1 = "GABA4_2", group.by = 'predicted.id')

In [29]:
GABA4_2.vsAllmarkers.mouse[order(GABA4_2.vsAllmarkers.mouse$avg_log2FC, decreasing = TRUE),]

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Nxph1,6.037261e-57,0.9007922,0.793,0.370,1.689527e-52
Chrm3,7.090453e-36,0.7740659,0.676,0.321,1.984263e-31
Col25a1,5.424006e-32,0.6923594,0.372,0.106,1.517908e-27
Eya2,2.196489e-58,0.6758696,0.309,0.042,6.146874e-54
Nkain3,2.646620e-26,0.6534226,0.564,0.263,7.406565e-22
Clstn2,6.400821e-25,0.6332676,0.638,0.349,1.791270e-20
Dscaml1,1.297304e-22,0.5788472,0.660,0.348,3.630505e-18
Kcnmb2,5.892801e-18,0.5554775,0.457,0.211,1.649100e-13
Sgcd,1.860058e-19,0.5155108,0.702,0.473,5.205372e-15


# 2. Explore marker genes, just macaque mouse for glut11

# 2.5 just macaque for gaba2_1 and 2_2 markers

In [11]:
# just macaque markers (look for gaba2_1 and gaba2_2)

filename1 = here(paste0('data/tidy_data/tables/macaque_marker_genes','2023-08-16','.csv'))
all.markers.macaque <- read.csv(filename1)


In [14]:
just_gaba2_1_macaque = all.markers.macaque[all.markers.macaque$cluster == 'GABA2_1',]
just_gaba2_1_macaque[order(just_gaba2_1_macaque$avg_log2FC, decreasing = TRUE),]

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
6524,2.432009e-35,1.0491516,0.969,0.363,1.335027e-30,GABA2_1,MIR34AHG
6523,1.501758e-39,1.0484559,0.846,0.191,8.243748e-35,GABA2_1,RORB
6528,2.160571e-28,0.9811850,0.938,0.415,1.186024e-23,GABA2_1,SDK1
6533,8.630554e-25,0.8943787,0.877,0.339,4.737657e-20,GABA2_1,RGS6
6550,4.782724e-20,0.8598084,0.846,0.370,2.625428e-15,GABA2_1,CLSTN2
6540,2.982783e-22,0.8356851,0.985,0.556,1.637369e-17,GABA2_1,CACNA2D3
6526,1.741740e-32,0.8259451,0.831,0.223,9.561109e-28,GABA2_1,SULF2
6531,4.036353e-26,0.8250089,0.877,0.349,2.215716e-21,GABA2_1,AC005906.2
6559,5.896716e-18,0.8086602,0.862,0.461,3.236943e-13,GABA2_1,CDH12


# 3. Explore cell type specific marker genes

In [1]:
# load previously generate marker genes
library(here)
filename1 = here(paste0('data/tidy_data/tables/xspecies_marker_genes','2023-08-16','.csv'))
filename2 = here(paste0('data/tidy_data/tables/xspecies_unique_marker_genes','2023-08-16','.csv'))
macaque_human_mouse <- read.csv(filename1)
macaque_human_mouse_specific <- read.csv(filename2)

filename3 = here(paste0('data/tidy_data/tables/macaque_mouse_marker_genes','2023-08-16','.csv'))
filename4 = here(paste0('data/tidy_data/tables/macaque_mouse_unique_marker_genes','2023-08-16','.csv'))
macaque_mouse <- read.csv(filename3)
macaque_mouse_specific <- read.csv(filename4)

Warning message:
“package ‘here’ was built under R version 4.1.3”
here() starts at /projects/pfenninggroup/singleCell/Macaque_SealDorsalHorn_snATAC-seq



In [77]:
### first get glut11
stringent_macaque_mouse = macaque_mouse_specific[macaque_mouse_specific$avg_log2FC.macaque > .5,]
stringent_macaque_mouse[stringent_macaque_mouse$cluster == 'GLUT11',]$gene

[1] "PBX3"  "SLIT2" "VWC2"

In [3]:
stringent = macaque_human_mouse_specific[macaque_human_mouse_specific$avg_log2FC > 0.6,]

In [48]:

stringent[stringent$cluster == 'GLUT1',]$gene

[1] "GABRG3" "HMGA2"

In [49]:
stringent[stringent$cluster == 'GLUT2',]$gene

[1] "C8orf34" "NMUR2"

In [50]:
stringent[stringent$cluster == 'GLUT3',]$gene

[1] "CDH10"  "GRID1"  "NMU"    "SEMA3C" "TAC3"

In [51]:
stringent[stringent$cluster == 'GLUT5',]$gene

[1] "KCND3"  "KCTD8"  "LMO3"   "OTOF"   "PCDH15" "PDE1C"

In [52]:
stringent[stringent$cluster == 'GLUT6',]$gene

character(0)

In [53]:
stringent[stringent$cluster == 'GLUT7',]$gene

[1] "CDH22" "MAF"

In [54]:
stringent[stringent$cluster == 'GLUT8',]$gene

[1] "ADCY2" "CHGB"

In [55]:
stringent[stringent$cluster == 'GLUT9',]$gene

[1] "NTS"

In [56]:
stringent[stringent$cluster == 'GLUT10',]$gene

[1] "KCNH8" "NR2F1" "PRKCG" "SNTB1"

In [57]:
stringent[stringent$cluster == 'GLUT11',]$gene

character(0)

In [ ]:
stringent[stringent$cluster == 'GABA1',]$gene

[1] "HCN1"    "TCF4"    "ZNF385B"

In [59]:
stringent[stringent$cluster == 'GABA2_1',]$gene

[1] "MGAT4C" "SORCS3" "SV2C"

In [60]:
stringent[stringent$cluster == 'GABA2_2',]$gene

[1] "RYR3"

In [61]:
stringent[stringent$cluster == 'GABA3',]$gene

character(0)

In [4]:
stringent[stringent$cluster == 'GABA4_1',]$gene

[1] "GPC5"

In [5]:
macaque_human_mouse_specific[macaque_human_mouse_specific$cluster == 'GABA4_1',]$gene

[1] "BCL11A"   "BRINP3"   "GPC5"     "LY6H"     "MEF2C"    "NETO2"    "SERPINE2"

In [62]:
stringent[stringent$cluster == 'GABA5',]$gene

[1] "ZEB2"

In [40]:
# GABA2_1
macaque_human_mouse_specific[macaque_human_mouse_specific$cluster == 'GABA1',]$gene

[1] "APBA2"   "ASAP1"   "BICC1"   "CDH20"   "CLMP"    "CRTAC1"  "ELAVL2" 
 [8] "FAM20A"  "GRM8"    "HCN1"    "PLCL1"   "PRKCQ"   "TCF4"    "TLL1"   
[15] "TMTC2"   "ZNF385B"

In [42]:
macaque_human_mouse_specific[macaque_human_mouse_specific$cluster == 'GABA5',]$gene

[1] "CDH6"  "DNER"  "LRP1B" "NWD2"  "PROX1" "RPH3A" "SCG2"  "SGCZ"  "STAC" 
[10] "ZEB2"

In [43]:
# GABA2_1
macaque_human_mouse_specific[macaque_human_mouse_specific$cluster == 'GABA2_1',]$gene

[1] "ATP2B2"   "CACNA1I"  "CAMTA1"   "COL19A1"  "DGKG"     "IL1RAPL1"
 [7] "MGAT4C"   "MSI2"     "NFIA"     "PRR16"    "RHBDL3"   "SORCS3"  
[13] "SV2C"     "SYN3"

In [44]:
# GABA2_2
macaque_human_mouse_specific[macaque_human_mouse_specific$cluster == 'GABA2_2',]$gene
macaque_human_mouse_specific[macaque_human_mouse_specific$cluster == 'GABA2_2',]

[1] "NETO1" "NOVA1" "RYR3"  "TENM2"

,gene,cluster,p_val.macaque,avg_log2FC.macaque,pct.1.macaque,pct.2.macaque,p_val_adj.macaque,p_val.human,avg_log2FC.human,pct.1.human,pct.2.human,p_val_adj.human,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
95,NETO1,GABA2_2,2.991175e-10,0.3497554,0.521,0.219,1.641976e-05,8.162515e-03,0.5113544,0.500,0.230,1.0000000,2.129765e-21,0.4931912,0.434,0.159,5.960147e-17
100,NOVA1,GABA2_2,8.680097e-09,0.4470763,0.803,0.563,4.764853e-04,2.033623e-05,0.9954634,0.929,0.852,0.4522371,1.277139e-08,0.3021760,0.400,0.210,3.574073e-04
129,RYR3,GABA2_2,5.299454e-22,0.7637845,0.944,0.509,2.909082e-17,9.589158e-06,1.2310913,0.929,0.520,0.2132437,4.245175e-47,0.7190427,0.920,0.516,1.188012e-42
154,TENM2,GABA2_2,2.362240e-10,0.3654246,1.000,0.854,1.296728e-05,4.024489e-04,1.3713604,0.929,0.862,1.0000000,3.388245e-70,0.5706557,0.989,0.780,9.482005e-66
